## 强化学习信息定义
值得注意的是,这是一个连续性的任务,而不是一个阶段性任务

### 状态
我们暂时先定义一个最简单的状态,后续可以增加更多复杂的环境因素
1. 执有股票数量:stock_hold_num
2. 剩余资金:fund_remain
3. 当前股票价格:stock_price
4. 市场股票总数:stock_total_num
5. 股票当前交易量:stock_exchange_num
6. 历史股票价格小跨度(时-分,用于捕获价格趋势,后续可以用长短记忆网络来捕获):stock_price(t-1)
7. 历史股票价格大跨度(年-天)stock_price_day(t-1)
8. 历史股票价格记忆长度:history_memory_len(t)
9. 初始资金:fund_init
10. 用户预期: e=1.01(如果收入的增长低于这个预期,用户将不愿意投入资金进来)
11. 交易费、手续费等其他费用: m=xxx
12. 用户预期增长周期: p=10(如果收入达到的用户的预期,这个时候用户的预期也会增长.如果没有在这个周期内达到用户的预期,用户也会放弃.即使第一次在周期内达到了,但第二个周期内没有达到,用户也会放弃)
13. 股票价格打折系数:b=0.9(在股市里面的钱,我们都不认为他是自己的钱,因为他随时可能会跌.提出来的时候,可能就少于当时的价格,所以我们需要对股市里面的钱进行打折)
14. 交易对于股市的影响,由于我们是小股东,所有暂时不考虑对于股市的影响,如果交易量太大则需要考虑

### 动作
我们需要AI进行股票交易,包括买入和卖出.其中买入和卖出的股票总数必须是100的整数,抛出的时候可以将剩余卖光,不需要是100的整数
1. 买入数量: buy_num
2. 卖出数量: sell_num

### 奖励
我们希望AI能够自己买卖股票来实现盈利,所以我们最终的目标是剩余资金fund最大化,但也不能不进行买卖,这样我们资金也不会增长.而且我们不能低于用户预期,这样将失去用户.并且每笔交易将产生固定的交易费用.如果我们没有在用户预期的周期内达到用户的预期,我们将失去用户,也就是说我们任务失败了.当一个周期结束后,下一个周期也一样.用户永远希望股票永久增长.当整体股市下降的时候,我们需要Ai止损,即使不能增长也不能输掉钱.其中股市不能套入太多的资金在里面,我们需要及时提出资金,也就是fund_remain要尽可能的大,但fund_remain大的时候,投入市场的股票也就少了,这个时候我们不能通过股票去赚钱,所以这个需要Ai去平衡.  
所以我们定义奖励如下:      
reward=fund_remain+stock_hold_num*stock_price*b-fund_init*e-m    
其中:   
fund_remain+stock_hold_num*stock_price*b 为当前拥有剩余资金+剩余股票数量的总价值*打折系数    
fund_init*e 为用户最低预期收入    
m 为将剩余的股票全部抛出需要的交易费用  
上面为最直接简单粗暴的方案.  
下面我们可以根据实际股票的增长来奖励惩罚Ai.也就是说股票上涨的时候,ai没有买入赚钱,我们惩罚他(赚钱的定义中需要去除交易费用,也就是赚的钱如果减去交易费用,返回亏本了,这样也惩罚).在股票下跌的时候,我们需要ai将股票卖出,避免亏损    
if(current_stock_price-last_stock_price)    
这种计算方式我们可以后续再考虑,我们先采用简单粗暴的方式  

## 环境
这里我们先用AlmgrenandChriss环境来模拟股票环境,后续我们可以通过爬虫接入炒股模拟软件的数据  
那么我们开始吧:

### AlmgrenandChriss环境
